In [29]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import SeleniumURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [ ]:
urls = []
for week in range(1,19):
    urls.append(f"https://www.espn.com/nfl/schedule/_/week/{week}/year/2023/seasontype/2")

loader = SeleniumURLLoader(urls=urls)
url_data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
all_text = text_splitter.split_documents(url_data)

In [40]:
embeddings = OpenAIEmbeddings() # Defaults to ada
docsearch = Chroma.from_documents(all_text, embeddings)
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model="gpt-3.5-turbo"), chain_type="stuff", retriever=docsearch.as_retriever())

In [41]:
print(qa.run("How many times does the colts play the bills"))

The Colts play the Bills once in the given schedule.


In [44]:
print(qa.run("How many times does miami play buffalo this year"))

I don't know the answer to that question.


In [46]:
from langchain.prompts import PromptTemplate
prompt_template = """
Instructions:
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Role: 
You are sports analyst given a NFL schedul with a list of games that are in the future and that have already happened. 
Your goal is to provide information on the NFL schedule.

Format of content:
Indianapolis \xa0 @ \xa0 Atlanta is  Indianapolis playing in Atlanta

Content:
{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model="gpt-3.5-turbo"), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)

In [47]:
print(qa.run("How many times does miami play buffalo this year"))

Miami does not play Buffalo this year.
